In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import silhouette_samples, silhouette_score
import math
import random
import matplotlib.ticker as mtick
import re

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsolutePercentageError


In [15]:
random.seed(42)

In [16]:
# Load the dataset
# df = pd.read_csv('/content/additional_data_cleaned_ER.csv')

# df_0 = pd.read_csv('/content/additional_data_cluster_0.csv')
df_1 = pd.read_csv('/content/additional_data_cluster_1.csv')
# df_2 = pd.read_csv('/content/additional_data_cluster_2.csv')
df = pd.concat([df_1], ignore_index=True)

# df = df.dropna()

# df['log_price'] = np.log(df['price'])
# df = df.drop(columns = 'price')


df.shape

(3392, 9)

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

In [19]:
X = df[[column for column in df.columns if column != 'price']]
y = df['price']

In [20]:
# Identify numerical and categorical columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# Define preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with the mean
    ('scaler', StandardScaler())  # Standardize numerical features
])

# Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [23]:
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)

X_train = X_train.toarray()
X_val = X_val.toarray()
X_test = X_test.toarray()

In [24]:
def r_squared(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - ss_res / (ss_tot + tf.keras.backend.epsilon())


# Very good result for df_2 with log_price
model = Sequential([
    Dense(120, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(100, activation='relu', kernel_initializer='he_normal'),
    Dense(80, activation='relu', kernel_initializer='he_normal'),
    Dense(60, activation='relu', kernel_initializer='he_normal'),
    Dense(40, activation='relu', kernel_initializer='he_normal'),
    Dense(20, activation='relu', kernel_initializer='he_normal'),
    Dense(1)
])

# Better result for df_0, normal price (no log_price, no drop_na)
# model = Sequential([
#     Dense(180, activation='relu', input_shape=(X_train.shape[1],)),
#     Dense(150, activation='relu', kernel_initializer='he_normal'),
#     Dense(120, activation='relu', kernel_initializer='he_normal'),
#     Dense(110, activation='relu', kernel_initializer='he_normal'),
#     Dense(100, activation='relu', kernel_initializer='he_normal'),
#     Dense(90, activation='relu', kernel_initializer='he_normal'),
#     Dense(80, activation='relu', kernel_initializer='he_normal'),
#     Dense(70, activation='relu', kernel_initializer='he_normal'),
#     Dense(60, activation='relu', kernel_initializer='he_normal'),
#     Dense(40, activation='relu', kernel_initializer='he_normal'),
#     Dense(20, activation='relu', kernel_initializer='he_normal'),
#     Dense(1)
# ])

model.compile(optimizer='adam', loss='mse', metrics=['mae', MeanSquaredError(), RootMeanSquaredError(), MeanAbsolutePercentageError(), r_squared])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping])

Epoch 1/100
68/68 [==============================] - 2s 8ms/step - loss: 445171924992.0000 - mae: 580769.2500 - mean_squared_error: 445171924992.0000 - root_mean_squared_error: 667212.0625 - mean_absolute_percentage_error: 99.8633 - r_squared: -3.3013 - val_loss: 481376894976.0000 - val_mae: 604477.5000 - val_mean_squared_error: 481376894976.0000 - val_root_mean_squared_error: 693813.3125 - val_mean_absolute_percentage_error: 98.8864 - val_r_squared: -3.3113
Epoch 2/100
68/68 [==============================] - 0s 5ms/step - loss: 267247288320.0000 - mae: 407186.3438 - mean_squared_error: 267247288320.0000 - root_mean_squared_error: 516959.6562 - mean_absolute_percentage_error: 69.9345 - r_squared: -1.5731 - val_loss: 56105504768.0000 - val_mae: 185216.5938 - val_mean_squared_error: 56105504768.0000 - val_root_mean_squared_error: 236866.0000 - val_mean_absolute_percentage_error: 41.8814 - val_r_squared: 0.5057
Epoch 3/100
68/68 [==============================] - 0s 5ms/step - loss: 3839

In [25]:
loss, mae, mse, rmse, mape, r2 = model.evaluate(X_val, y_val)

17/17 [==============================] - 0s 3ms/step - loss: 20914139136.0000 - mae: 95903.3828 - mean_squared_error: 20914139136.0000 - root_mean_squared_error: 144617.2188 - mean_absolute_percentage_error: 17.1598 - r_squared: 0.8144


In [26]:
rmse/(y_val.max() - y_val.min())

0.10329801339285714